In [25]:
import mediapipe as mp
import cv2
import numpy as np

In [26]:
mp_drawing=mp.solutions.drawing_utils
mp_drawing_styles=mp.solutions.drawing_styles
mp_pose=mp.solutions.pose

In [3]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    
    cv2.imshow('MediaPipe pose', cv2.flip(image, 1))

    k = cv2.waitKey(1) & 0xFF
    if(k == 27):
        break
cap.release()
cv2.destroyAllWindows()

## Save the video

In [49]:
cap=cv2.VideoCapture(0)

height=cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width=cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps=cap.get(cv2.CAP_PROP_FPS)

videoWritter=cv2.VideoWriter('video_data\stance1.avi',cv2.VideoWriter_fourcc('P','I','M','1'),fps,( int(width) , int(height) ))

while cap.isOpened():
    ret,frame=cap.read()

    try:
        cv2.imshow('pose',frame)
        videoWritter.write(frame)
    except Exception as e:
        break

    k = cv2.waitKey(1) & 0xFF
    if(k == 27):
        break
cap.release()
videoWritter.release()
cv2.destroyAllWindows()

## Exporting

In [50]:
import csv
import os
import numpy as np
import matplotlib.pyplot as plt

In [51]:
landmarks=['class']
for i in range(1,33+1):
    landmarks+= ['x{}'.format(i), 'y{}'.format(i), 'z{}'.format(i), 'v{}'.format(i)]

In [52]:
with open('stance_coords.csv',mode='w',newline='') as f:
    csv_writer=csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [53]:
def export_landmarks(results,stance):
    try:
        keypoints=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()
        action_array = np.array([stance])
        keypoints = np.concatenate((action_array, keypoints))

        with open('stance_coords.csv',mode='a',newline='') as f:
            csv_writer=csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        pass

In [54]:
cap=cv2.VideoCapture('stance1.avi')

with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      continue
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    
    cv2.imshow('pose',image)

    k = cv2.waitKey(1) & 0xFF
    if(k==97):  #a
       export_landmarks(results,'wide')
    if(k==115):  #s
       export_landmarks(results,'neutral')
    if(k==100): #d
       export_landmarks(results,'narrow')

    if(k == 27):
        break
cap.release()
cv2.destroyAllWindows()

## Training Model

In [55]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [61]:
df=pd.read_csv('stance_coords.csv')

In [62]:
df['class'].value_counts()

class
narrow     366
wide       285
neutral    222
Name: count, dtype: int64

In [58]:
df.head(2)

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,wide,0.486891,0.161831,-0.410740,0.999994,0.495311,0.144539,-0.389651,0.999981,0.500011,...,0.263976,0.877584,0.679821,0.944764,0.086859,0.979125,0.302998,0.948523,0.095927,0.982755
1,wide,0.488632,0.171418,-0.392792,0.999995,0.497058,0.153132,-0.370833,0.999986,0.501180,...,0.270792,0.885091,0.683959,0.945675,0.082152,0.980912,0.304800,0.946145,0.105384,0.984597


In [63]:
x=df.iloc[:,1:]
y=df.iloc[:,0]

### handling imbalaced

In [64]:
from imblearn.over_sampling import SMOTE
x,y=SMOTE(random_state=32).fit_resample(x,y)

In [66]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=32,stratify=y)

In [67]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

In [68]:
pipelines={
    'rf': make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb': make_pipeline(StandardScaler(),GradientBoostingClassifier())
}

In [69]:
fit_models={}
for algo,pipeline in pipelines.items():
    model=pipeline.fit(x_train,y_train)
    fit_models[algo]=model

In [70]:
landmarks=[]
for val in range(1,33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [71]:
keypoints=np.array([0.4360373616218567,0.26478785276412964,-0.6607211828231812,0.9999974966049194,0.4441344738006592,0.24916169047355652,-0.6336907148361206,0.9999911189079285,0.4497406482696533,0.24935220181941986,-0.6336749792098999,0.9999901652336121,0.45456892251968384,0.2496332973241806,-0.6337651610374451,0.9999900460243225,0.42742958664894104,0.24962657690048218,-0.6321507692337036,0.9999898076057434,0.422477662563324,0.25000011920928955,-0.6322957277297974,0.9999892115592957,0.41860318183898926,0.25014498829841614,-0.6323496699333191,0.9999893307685852,0.4626897871494293,0.257098525762558,-0.4822930693626404,0.999975323677063,0.4136722981929779,0.2581169009208679,-0.480527400970459,0.9999681711196899,0.4476795196533203,0.28475770354270935,-0.6052373051643372,0.9999968409538269,0.4274052083492279,0.28452789783477783,-0.6043713092803955,0.9999964833259583,0.5076010823249817,0.3685709536075592,-0.3927532136440277,0.9999759197235107,0.376088410615921,0.3795548379421234,-0.38380318880081177,0.999902069568634,0.516706109046936,0.5268847942352295,-0.4084544777870178,0.9889070391654968,0.3794488310813904,0.543158233165741,-0.4005517363548279,0.9828525185585022,0.5124207735061646,0.6685745120048523,-0.4906157851219177,0.9651187062263489,0.3756535053253174,0.6928625702857971,-0.4866250157356262,0.9422730207443237,0.5174915194511414,0.7059602737426758,-0.5309696197509766,0.9251753687858582,0.3713611662387848,0.7310739755630493,-0.5307987928390503,0.887118935585022,0.5071402192115784,0.7081703543663025,-0.5543004274368286,0.928005039691925,0.3855089545249939,0.7290276288986206,-0.5596837997436523,0.890435516834259,0.5039002895355225,0.6989316344261169,-0.5006574988365173,0.9154057502746582,0.38834574818611145,0.7185326218605042,-0.5005630254745483,0.8736847043037415,0.4907452464103699,0.5520334243774414,-0.0017842678353190422,0.9999696612358093,0.41634097695350647,0.559161365032196,0.001531944377347827,0.9999473690986633,0.5502486824989319,0.6607305407524109,-0.475755512714386,0.9916160106658936,0.3557720184326172,0.6755240559577942,-0.4120989441871643,0.9881277084350586,0.5320411324501038,0.7981935143470764,-0.0866389125585556,0.9874908328056335,0.3832264542579651,0.8041484951972961,-0.048621080815792084,0.9890920519828796,0.5232718586921692,0.8167591094970703,-0.055673450231552124,0.9150554537773132,0.3938315510749817,0.8190670609474182,-0.019249562174081802,0.8985493779182434,0.5489282608032227,0.8703070878982544,-0.21380701661109924,0.979395866394043,0.3633933663368225,0.8664740920066833,-0.1735038012266159,0.9785320162773132])
data=pd.DataFrame([keypoints],columns=landmarks)

In [72]:
fit_models['rf'].predict_proba(data)[0][0]

0.31

In [73]:
fit_models['rf'].predict(data)

array(['neutral'], dtype=object)

In [74]:
from sklearn.metrics import accuracy_score,recall_score,precision_score
import pickle

In [75]:
for algo,model in fit_models.items():
    y_pred=model.predict(x_test)
    print(algo,
          accuracy_score(y_test.values,y_pred),
          recall_score(y_test.values,y_pred,average='weighted'),
          precision_score(y_test.values,y_pred,average='weighted'))

rf 1.0 1.0 1.0
gb 1.0 1.0 1.0


In [76]:
pickle.dump(fit_models,open('stance_model.pkl','wb'))